# NAME : VANSH KOLTE
# ROLLno:62
# Compiler Design
# Practical_5

In [10]:
from prettytable import PrettyTable


def removeLeftRecursion(rulesDiction):

    store = {}
    for lhs in rulesDiction:
        alphaRules = []
        betaRules = []
        allrhs = rulesDiction[lhs]
        for subrhs in allrhs:
            if subrhs[0] == lhs:
                alphaRules.append(subrhs[1:])
            else:
                betaRules.append(subrhs)
        if alphaRules:
            lhs_ = lhs + "'"
            while lhs_ in rulesDiction or lhs_ in store:
                lhs_ += "'"
            for b in range(len(betaRules)):
                betaRules[b].append(lhs_)
            rulesDiction[lhs] = betaRules
            for a in range(len(alphaRules)):
                alphaRules[a].append(lhs_)
            alphaRules.append(['#'])
            store[lhs_] = alphaRules
    for left in store:
        rulesDiction[left] = store[left]
    return rulesDiction


def LeftFactoring(rulesDiction):
    newDict = {}
    for lhs in rulesDiction:
        allrhs = rulesDiction[lhs]
        temp = {}
        for subrhs in allrhs:
            if subrhs[0] not in temp:
                temp[subrhs[0]] = [subrhs]
            else:
                temp[subrhs[0]].append(subrhs)
        new_rule = []
        tempo_dict = {}
        for term_key in temp:
            allStartingWithTermKey = temp[term_key]
            if len(allStartingWithTermKey) > 1:
                lhs_ = lhs + "'"
                while lhs_ in rulesDiction or lhs_ in tempo_dict:
                    lhs_ += "'"
                new_rule.append([term_key, lhs_])
                tempo_dict[lhs_] = [g[1:] for g in allStartingWithTermKey]
            else:
                new_rule.append(allStartingWithTermKey[0])
        newDict[lhs] = new_rule
        for key in tempo_dict:
            newDict[key] = tempo_dict[key]
    return newDict


def first(rule):
    global diction, firsts
    if len(rule) == 0:
        return []
    if rule[0] in term_userdef:
        return [rule[0]]
    if rule[0] == '#':
        return ['#']
    if rule[0] in diction:
        fres = []
        for rhs in diction[rule[0]]:
            indivRes = first(rhs)
            for i in indivRes:
                if i not in fres:
                    fres.append(i)
            if '#' in indivRes:
                fres.remove('#')
                if len(rule) > 1:
                    fres += first(rule[1:])
        return fres
    return []


def follow(nt):
    global start_symbol, diction, follows
    solset = set()
    if nt == start_symbol:
        solset.add('$')
    for curNT in diction:
        for subrule in diction[curNT]:
            while nt in subrule:
                index_nt = subrule.index(nt)
                subrule = subrule[index_nt + 1:]
                if len(subrule) != 0:
                    res = first(subrule)
                    if '#' in res:
                        res.remove('#')
                        solset.update(res)
                        solset.update(follow(curNT))
                    else:
                        solset.update(res)
                else:
                    if nt != curNT:
                        solset.update(follow(curNT))
    return list(solset)


def computeAllFirsts():
    global diction, firsts
    for rule in rules:
        k = rule.split("->")
        lhs = k[0].strip()
        rhs = k[1].strip().split('|')
        diction[lhs] = [r.strip().split() for r in rhs]
    for y in diction:
        firsts[y] = set()
        for sub in diction[y]:
            res = first(sub)
            firsts[y].update(res)


def computeAllFollows():
    global follows
    for NT in diction:
        follows[NT] = set(follow(NT))


def createParseTable():
    global diction, firsts, follows, term_userdef
    terminals = term_userdef + ['$']
    mat = [['' for _ in range(len(terminals))] for _ in range(len(diction))]
    grammar_is_LL = True
    for lhs in diction:
        rhs = diction[lhs]
        for y in rhs:
            res = first(y)
            if '#' in res:
                res.remove('#')
                for f in res:
                    xnt = list(diction.keys()).index(lhs)
                    yt = terminals.index(f)
                    if mat[xnt][yt] == '':
                        mat[xnt][yt] = f"{lhs} -> {' '.join(y)}"
                    else:
                        grammar_is_LL = False
                        mat[xnt][yt] += f", {lhs} -> {' '.join(y)}"
                for f in follows[lhs]:
                    xnt = list(diction.keys()).index(lhs)
                    yt = terminals.index(f)
                    if mat[xnt][yt] == '':
                        mat[xnt][yt] = f"{lhs} -> {' '.join(y)}"
                    else:
                        grammar_is_LL = False
                        mat[xnt][yt] += f", {lhs} -> {' '.join(y)}"
            else:
                for f in res:
                    xnt = list(diction.keys()).index(lhs)
                    yt = terminals.index(f)
                    if mat[xnt][yt] == '':
                        mat[xnt][yt] = f"{lhs} -> {' '.join(y)}"
                    else:
                        grammar_is_LL = False
                        mat[xnt][yt] += f", {lhs} -> {' '.join(y)}"
    return mat, grammar_is_LL, terminals


def validateStringUsingStackBuffer(parsing_table, grammarll1, tabTerm,
                                   input_string):
    print(f"\nValidate String => {input_string}\n")
    if not grammarll1:
        return f"\nInput String = \"{input_string}\"\nGrammar is not LL(1)"

    stack = [start_symbol, '$']
    buffer = ['$'] + input_string.split()[::-1]
    print("{:>20} {:>20} {:>20}".format("Buffer", "Stack", "Action"))

    while True:
        if stack == ['$'] and buffer == ['$']:
            print("{:>20} {:>20} {:>20}".format(' '.join(buffer),
                                                ' '.join(stack), "Valid"))
            return "\nValid String!"
        elif stack[0] not in term_userdef:
            x = list(diction.keys()).index(stack[0])
            y = tabTerm.index(buffer[-1])
            if parsing_table[x][y] != '':
                entry = parsing_table[x][y]
                print("{:>20} {:>20} {:>25}".format(
                    ' '.join(buffer), ' '.join(stack),
                    f"T[{stack[0]}][{buffer[-1]}] = {entry}"))
                lhs_rhs = entry.split("->")
                lhs_rhs[1] = lhs_rhs[1].replace('#', '').strip()
                entryrhs = lhs_rhs[1].split()
                stack = entryrhs + stack[1:]
            else:
                return f"\nInvalid String! No rule at Table[{stack[0]}][{buffer[-1]}]."
        else:
            if stack[0] == buffer[-1]:
                print("{:>20} {:>20} {:>20}".format(' '.join(buffer),
                                                    ' '.join(stack),
                                                    f"Matched: {stack[0]}"))
                buffer = buffer[:-1]
                stack = stack[1:]
            else:
                return "\nInvalid String! Unmatched terminal symbols"


def displayFirsts():
    print("\nFIRST sets:")
    for nt in firsts:
        print(f"FIRST({nt}) = {{ {', '.join(firsts[nt])} }}")


def displayParsingTable(parsing_table, terminals):
    from prettytable import PrettyTable

    print("\nParsing Table:")
    x = PrettyTable()
    x.field_names = ["Non-Terminal"] + terminals

    for i, lhs in enumerate(diction.keys()):
        row = [lhs] + [
            parsing_table[i][j] if parsing_table[i][j] != '' else '-'
            for j in range(len(terminals))
        ]
        x.add_row(row)

    print(x)


# DRIVER CODE - MAIN

rules = [
    "S -> NP VP", "NP -> P | PN | D N", "VP -> V NP",
    "N -> championship | ball | toss", "V -> is | want | won | played",
    "P -> me | I | you", "PN -> India | Australia | Steve | John",
    "D -> the | a | an"
]

nonterm_userdef = ['S', 'NP', 'VP', 'N', 'V', 'P', 'PN', 'D']
term_userdef = [
    "championship", "ball", "toss", "is", "want", "won", "played", "me", "I",
    "you", "India", "Australia", "Steve", "John", "the", "a", "an"
]

diction = {}
firsts = {}
follows = {}

# Compute Firsts and Follows
computeAllFirsts()
start_symbol = list(diction.keys())[0]
computeAllFollows()

# Create Parsing Table
parsing_table, result, tabTerm = createParseTable()

# Display First sets
displayFirsts()

# Display Parsing Table
displayParsingTable(parsing_table, tabTerm)

# Input string manually
sample_input_string = input(
    "\nEnter a string to validate (e.g., 'India won the championship'): ")

# Validate the input string using stack buffer
if sample_input_string:
    validity = validateStringUsingStackBuffer(parsing_table, result, tabTerm,
                                              sample_input_string)
    print(validity)
else:
    print("\nNo input String detected")

 
 
 
 


FIRST sets:
FIRST(S) = { John, me, a, Steve, the, you, India, Australia, an, I }
FIRST(NP) = { John, me, a, Steve, the, you, India, Australia, an, I }
FIRST(VP) = { won, played, want, is }
FIRST(N) = { championship, toss, ball }
FIRST(V) = { won, played, want, is }
FIRST(P) = { me, you, I }
FIRST(PN) = { India, Steve, John, Australia }
FIRST(D) = { an, the, a }

Parsing Table:
+--------------+-------------------+-----------+-----------+------------+------------+------------+-------------+------------+------------+------------+-------------+-----------------+-------------+------------+------------+------------+------------+---+
| Non-Terminal |    championship   |    ball   |    toss   |     is     |    want    |    won     |    played   |     me     |     I      |    you     |    India    |    Australia    |    Steve    |    John    |    the     |     a      |     an     | $ |
+--------------+-------------------+-----------+-----------+------------+------------+------------+----------


Enter a string to validate (e.g., 'India won the championship'):  India won the championship



Validate String => India won the championship

              Buffer                Stack               Action
$ championship the won India                  S $  T[S][India] = S -> NP VP
$ championship the won India              NP VP $   T[NP][India] = NP -> PN
$ championship the won India              PN VP $ T[PN][India] = PN -> India
$ championship the won India           India VP $       Matched: India
$ championship the won                 VP $   T[VP][won] = VP -> V NP
$ championship the won               V NP $      T[V][won] = V -> won
$ championship the won             won NP $         Matched: won
  $ championship the                 NP $    T[NP][the] = NP -> D N
  $ championship the                D N $      T[D][the] = D -> the
  $ championship the              the N $         Matched: the
      $ championship                  N $ T[N][championship] = N -> championship
      $ championship       championship $ Matched: championship
                   $                    